In [1]:
# imports
import torch
from vggt.models.vggt import VGGT
from vggt.utils.load_fn import load_and_preprocess_images

device = "cuda" if torch.cuda.is_available() else "cpu"
# bfloat16 is supported on Ampere GPUs (Compute Capability 8.0+) 
dtype = torch.bfloat16 if torch.cuda.get_device_capability()[0] >= 8 else torch.float16

/athenahomes/yashsb/miniforge3/envs/py3d/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/athenahomes/yashsb/miniforge3/envs/py3d/lib/python3.8/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
# model loading
model = VGGT()
_URL = "https://huggingface.co/facebook/VGGT-1B/resolve/main/model.pt"
model.load_state_dict(torch.hub.load_state_dict_from_url(_URL))
model = model.to(device)

In [3]:
# load images to run the model on:
image_names = [f"examples/kitchen/images/{i:02d}.png" for i in range(0, 25, 5)]
images = load_and_preprocess_images(image_names).to(device)

In [ ]:
# cross-frame patch-to-patch attention matrix (with frame separators)
from importlib import reload
import vggt.utils.attn_capture as attn_capture
reload(attn_capture)
from vggt.utils.attn_capture import capture_and_save_patch_grid_attention

image_set_name = "kitchen"

# Downsample the patch grid for tractable visualization
# stride_h/stride_w of 4 means we sample every 4th patch row/col
saved_patchgrid = capture_and_save_patch_grid_attention(
    model,
    images,
    image_set_name=image_set_name,
    output_dir="attn_outputs",
    stride_h=4,
    stride_w=4,
    amp_dtype=dtype,
    max_query_chunk=2048,
)
saved_patchgrid


In [ ]:
# reload updated attention capture module so changes take effect
from importlib import reload
import vggt.utils.attn_capture as attn_capture
reload(attn_capture)
from vggt.utils.attn_capture import capture_and_save_global_attention


In [ ]:
# capture and save global attention maps
from vggt.utils.attn_capture import capture_and_save_global_attention

# choose a descriptive image set name
image_set_name = "kitchen"

# with torch.no_grad():
#     with torch.cuda.amp.autocast(dtype=dtype):
saved_paths = capture_and_save_global_attention(
    model,
    images,
    image_set_name=image_set_name,
    output_dir="attn_outputs",
    amp_dtype=dtype,
)

saved_paths


In [ ]:
# first-to-all attention visualization (frame 0 queries to all frames' patches)
from importlib import reload
import vggt.utils.attn_capture as attn_capture
reload(attn_capture)
from vggt.utils.attn_capture import capture_and_save_first_to_all_attention

image_set_name = "kitchen"

saved_first2all = capture_and_save_first_to_all_attention(
    model,
    images,
    image_set_name=image_set_name,
    output_dir="attn_outputs_first2all",
    stride_h=1,
    stride_w=1,
    amp_dtype=dtype,
    max_query_chunk=2048,
)
saved_first2all
